# Module 2: Initial Price Push (Daily 6 AM Reset)

## Purpose
This module runs once daily at 6 AM Cairo time to:
1. Reset prices for all SKUs based on ABC classification
2. Set initial cart rules based on normal_refill and stddev
3. Apply status-based adjustments (combined_status + yesterday_status)

## Price Setting Logic
- **Zero demand SKUs**: Market minimum price + SKU discount
- **With market data**: A=25th percentile, B=50th, C=75th
- **Without market data**: A=50% margin range, B=75%, C=90%
- **No data SKUs**: Average margin of their category

## Status Adjustment Logic
- Both below On Track: -1 step from target
- Both above On Track: +1 step from target
- Combined lower, Yesterday higher: No action
- Combined higher, Yesterday lower: -1 step from target
- On Track: No action


In [ ]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import sys
sys.path.append('..')

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()
CURRENT_HOUR = CAIRO_NOW.hour

# Configuration constants
ABC_MARKET_PERCENTILES = {'A': 25, 'B': 50, 'C': 75}
ABC_MARGIN_PERCENTILES = {'A': 50, 'B': 75, 'C': 90}
ABC_CART_STD_MULTIPLIERS = {'A': 1, 'B': 2, 'C': 5}
MIN_CART_RULE = 2
MAX_CART_RULE = 150
MIN_PRICE_REDUCTION_PCT = 0.0025
STATUS_BELOW_ON_TRACK = ['No Data', 'Critical', 'Struggling', 'Underperforming']
STATUS_ABOVE_ON_TRACK = ['Over Achiever', 'Star Performer']
STATUS_ON_TRACK = ['On Track']

# Input/Output files
INPUT_FILE = '../pricing_with_discount.xlsx'
OUTPUT_FILE = f'module_2_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'
ZERO_DEMAND_SKU_DISCOUNT = 5  # 5% discount

print(f"Module 2: Initial Price Push")
print(f"Run Time (Cairo): {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Input: {INPUT_FILE}")
print(f"Output: {OUTPUT_FILE}")


In [ ]:
# =============================================================================
# LOAD DATA
# =============================================================================
print("Loading data from Module 1 output...")
df = pd.read_excel(INPUT_FILE)
print(f"Loaded {len(df)} records")
print(f"\nABC Class Distribution:")
print(df['abc_class'].value_counts().to_string() if 'abc_class' in df.columns else "No abc_class column")


In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def is_below_on_track(status):
    """Check if status is below On Track."""
    return str(status).strip() in STATUS_BELOW_ON_TRACK

def is_above_on_track(status):
    """Check if status is above On Track."""
    return str(status).strip() in STATUS_ABOVE_ON_TRACK

def is_on_track(status):
    """Check if status is On Track."""
    return str(status).strip() in STATUS_ON_TRACK

def calculate_margin(price, wac):
    """Calculate margin from price and WAC."""
    if pd.isna(price) or pd.isna(wac) or price == 0:
        return None
    return (price - wac) / price

def get_market_tiers(row):
    """Get sorted list of market price tiers."""
    tiers = []
    for col in ['minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum']:
        val = row.get(col)
        if pd.notna(val) and val > 0:
            tiers.append(val)
    return sorted(set(tiers))

def get_margin_tiers(row):
    """Get sorted list of margin-based price tiers (converted to prices)."""
    tiers = []
    wac = row.get('wac_p', 0)
    if wac <= 0:
        return tiers
    
    for tier_col in ['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
                     'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']:
        margin = row.get(tier_col)
        if pd.notna(margin) and 0 < margin < 1:
            price = wac / (1 - margin)
            tiers.append(round(price, 2))
    return sorted(set(tiers))

def find_next_price_above(current_price, row):
    """
    Find the first price tier ABOVE current_price.
    Priority: Market tiers first, then margin tiers.
    Returns current_price if nothing found.
    """
    if pd.isna(current_price) or current_price <= 0:
        return current_price
    
    # Try market tiers first
    market_tiers = get_market_tiers(row)
    for tier in market_tiers:
        if tier > current_price:
            return round(tier, 2)
    
    # Try margin tiers
    margin_tiers = get_margin_tiers(row)
    for tier in margin_tiers:
        if tier > current_price:
            return round(tier, 2)
    
    # Nothing found above, keep current
    return current_price

def find_next_price_below(current_price, row):
    """
    Find the first price tier BELOW current_price.
    Priority: Market tiers first, then margin tiers.
    Returns current_price if nothing found.
    """
    if pd.isna(current_price) or current_price <= 0:
        return current_price
    
    # Try market tiers first (reverse order to find highest below)
    market_tiers = get_market_tiers(row)
    for tier in reversed(market_tiers):
        if tier < current_price:
            return round(tier, 2)
    
    # Try margin tiers (reverse order)
    margin_tiers = get_margin_tiers(row)
    for tier in reversed(margin_tiers):
        if tier < current_price:
            return round(tier, 2)
    
    # Nothing found below, keep current
    return current_price

print("Helper functions loaded.")


In [ ]:
# =============================================================================
# STATUS ADJUSTMENT & PRICE FUNCTIONS
# =============================================================================

def get_price_action(combined_status, yesterday_status):
    """
    Determine price action based on status combination.
    
    Returns:
        str: 'increase', 'decrease', or 'hold'
        str: Reason for action
    """
    combined_below = is_below_on_track(combined_status)
    combined_above = is_above_on_track(combined_status)
    combined_on = is_on_track(combined_status)
    yesterday_below = is_below_on_track(yesterday_status)
    yesterday_above = is_above_on_track(yesterday_status)
    yesterday_on = is_on_track(yesterday_status)
    
    # On Track = no action
    if combined_on:
        return 'hold', "On Track - no price change"
    
    # Both above On Track: INCREASE price (go to first tier above current)
    if combined_above and (yesterday_above or yesterday_on):
        return 'increase', f"Both above/on ({combined_status}, {yesterday_status}) - increase"
    
    # Both below On Track: DECREASE price (go to first tier below current)
    if combined_below and (yesterday_below or yesterday_on):
        return 'decrease', f"Both below/on ({combined_status}, {yesterday_status}) - decrease"
    
    # Combined below, Yesterday above: No action (oscillation prevention)
    if combined_below and yesterday_above:
        return 'hold', f"Oscillation prevention ({combined_status} vs {yesterday_status}) - hold"
    
    # Combined above, Yesterday below: DECREASE (stabilize)
    if combined_above and yesterday_below:
        return 'decrease', f"Stabilizing ({combined_status} vs {yesterday_status}) - decrease"
    
    return 'hold', "Default - no price change"

def apply_price_action(current_price, action, row):
    """
    Apply price action: find next tier above/below current price.
    
    Args:
        current_price: Current SKU price
        action: 'increase', 'decrease', or 'hold'
        row: DataFrame row with tier data
    
    Returns:
        float: New price
        str: Source of new price (market/margin/unchanged)
    """
    if action == 'hold' or pd.isna(current_price):
        return current_price, 'unchanged'
    
    if action == 'increase':
        new_price = find_next_price_above(current_price, row)
        if new_price > current_price:
            # Determine source
            market_tiers = get_market_tiers(row)
            source = 'market' if new_price in market_tiers else 'margin'
            return new_price, source
        return current_price, 'unchanged (no tier above)'
    
    if action == 'decrease':
        new_price = find_next_price_below(current_price, row)
        if new_price < current_price:
            # Determine source
            market_tiers = get_market_tiers(row)
            source = 'market' if new_price in market_tiers else 'margin'
            
            # Apply commercial minimum floor
            commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            
            return new_price, source
        return current_price, 'unchanged (no tier below)'
    
    return current_price, 'unchanged'

# Cart rule multipliers based on ABC class + yesterday status
CART_MULTIPLIERS = {
    # (abc_class, status_category) -> multiplier
    ('A', 'above'): 2,
    ('A', 'below'): 5,
    ('A', 'on_track'): 3,
    ('B', 'above'): 2,
    ('B', 'below'): 7,
    ('B', 'on_track'): 5,
    ('C', 'above'): 5,
    ('C', 'below'): 10,
    ('C', 'on_track'): 7,
}

def get_cart_multiplier(abc_class, yesterday_status):
    """Get cart rule multiplier based on ABC class and yesterday status."""
    abc = str(abc_class).upper()
    if abc not in ['A', 'B', 'C']:
        abc = 'C'
    
    if is_above_on_track(yesterday_status):
        status_cat = 'above'
    elif is_below_on_track(yesterday_status):
        status_cat = 'below'
    else:
        status_cat = 'on_track'
    
    return CART_MULTIPLIERS.get((abc, status_cat), 7)  # Default to 7

def get_initial_cart_rule(row, is_oos=False, is_zero_demand=False):
    """
    Calculate initial cart rule based on ABC class + yesterday performance.
    
    Multipliers:
    - A + above: 2, A + below: 5, A + on_track: 3
    - B + above: 2, B + below: 7, B + on_track: 5
    - C + above: 5, C + below: 10, C + on_track: 7
    
    Special cases:
    - OOS: normal_refill + 3*std (min 2)
    - Zero Demand: normal_refill + 10*std (open cart)
    """
    normal_refill = row.get('normal_refill', 5)
    stddev = row.get('refill_stddev', 2)
    
    # Special case: OOS - set to refill + 3*std
    if is_oos:
        target_cart = normal_refill + (3 * stddev)
        return max(MIN_CART_RULE, min(MAX_CART_RULE, int(target_cart)))
    
    # Special case: Zero Demand - open cart to refill + 10*std
    if is_zero_demand:
        target_cart = normal_refill + (10 * stddev)
        return max(MIN_CART_RULE, min(MAX_CART_RULE, int(target_cart)))
    
    # Normal case: use ABC class + yesterday status multiplier
    abc_class = row.get('abc_class', 'C')
    yesterday_status = row.get('yesterday_status', 'No Data')
    
    multiplier = get_cart_multiplier(abc_class, yesterday_status)
    target_cart = normal_refill + (multiplier * stddev)
    
    return max(MIN_CART_RULE, min(MAX_CART_RULE, int(target_cart)))

print("Status adjustment and price functions loaded.")


In [ ]:
# =============================================================================
# MAIN ENGINE: GENERATE INITIAL PRICE PUSH
# =============================================================================

def get_max_price(row):
    """Get maximum price: market_max first, then highest margin tier."""
    # Try market max first
    market_max = row.get('maximum')
    if pd.notna(market_max) and market_max > 0:
        return market_max, 'market_max'
    
    # Fallback: highest margin tier
    margin_tiers = get_margin_tiers(row)
    if margin_tiers:
        return margin_tiers[-1], 'margin_max'  # Last = highest
    
    # Fallback: current price
    return row.get('current_price', 0), 'unchanged'

def find_price_n_steps_below(current_price, n_steps, row):
    """Find price N steps below current (iteratively find next tier below)."""
    price = current_price
    for _ in range(n_steps):
        next_price = find_next_price_below(price, row)
        if next_price >= price:  # No tier below found
            break
        price = next_price
    return price

def generate_initial_price_push(row):
    """
    Generate initial price push action for a single SKU.
    
    Logic:
    - Stocks = 0: Set to market_max or margin_max (highest price)
    - Zero demand + yesterday below on track: Go 2 steps below current
    - Zero demand + yesterday above on track: Keep current price
    - Otherwise: Adjust price relative to CURRENT price based on status
    """
    result = {
        'product_id': row.get('product_id'),
        'warehouse_id': row.get('warehouse_id'),
        'sku': row.get('sku'),
        'brand': row.get('brand'),
        'cat': row.get('cat'),
        'abc_class': row.get('abc_class', 'C'),
        'current_price': row.get('current_price'),
        'wac_p': row.get('wac_p'),
        'stocks': row.get('stocks', 0),
        'combined_status': row.get('combined_status'),
        'yesterday_status': row.get('yesterday_status'),
        'zero_demand': row.get('zero_demand', 0),
        'sensitivity': row.get('sensitivity', row.get('product_sensitivity')),
        'new_price': None,
        'new_cart_rule': None,
        'new_margin': None,
        'current_margin': None,
        'sku_discount': 0,
        'price_source': None,
        'price_action': None,
        'price_reason': None,
    }
    
    wac = row.get('wac_p', 0)
    current_price = row.get('current_price', 0)
    result['current_margin'] = calculate_margin(current_price, wac)
    yesterday_status = row.get('yesterday_status', 'No Data')
    
    # CASE 1: Out of Stock (stocks = 0) - Set to MAX price
    if row.get('stocks', 0) == 0:
        max_price, price_source = get_max_price(row)
        result['new_price'] = max_price
        result['price_source'] = price_source
        result['price_action'] = 'oos_max'
        result['price_reason'] = 'OOS - set to max price'
        result['new_cart_rule'] = get_initial_cart_rule(row, is_oos=True)  # OOS cart: refill + 3*std
        result['new_margin'] = calculate_margin(max_price, wac)
        return result
    
    # CASE 2: Zero Demand SKUs (has stock but no recent sales)
    if row.get('zero_demand', 0) == 1:
        yesterday_below = is_below_on_track(yesterday_status)
        yesterday_above = is_above_on_track(yesterday_status)
        
        if yesterday_below:
            # Yesterday below on track: Go 2 steps below current price
            new_price = find_price_n_steps_below(current_price, 2, row)
            
            # Apply commercial minimum floor
            commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            
            result['new_price'] = new_price
            result['price_source'] = 'market' if new_price in get_market_tiers(row) else 'margin'
            result['price_action'] = 'zero_demand_decrease'
            result['price_reason'] = f'Zero demand + yesterday below ({yesterday_status}) - 2 steps below'
            result['sku_discount'] = ZERO_DEMAND_SKU_DISCOUNT
        elif yesterday_above:
            # Yesterday above on track: Keep current price
            result['new_price'] = current_price
            result['price_source'] = 'unchanged'
            result['price_action'] = 'zero_demand_hold'
            result['price_reason'] = f'Zero demand + yesterday above ({yesterday_status}) - keep current'
            result['sku_discount'] = ZERO_DEMAND_SKU_DISCOUNT
        else:
            # Yesterday on track or no data: Go 1 step below
            new_price = find_next_price_below(current_price, row)
            commercial_min = row.get('commercial_min_price', row.get('minimum', 0))
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            
            result['new_price'] = new_price
            result['price_source'] = 'market' if new_price in get_market_tiers(row) else 'margin'
            result['price_action'] = 'zero_demand_decrease'
            result['price_reason'] = f'Zero demand + yesterday on track ({yesterday_status}) - 1 step below'
            result['sku_discount'] = ZERO_DEMAND_SKU_DISCOUNT
        
        result['new_cart_rule'] = get_initial_cart_rule(row, is_zero_demand=True)  # Zero demand: refill + 10*std
        result['new_margin'] = calculate_margin(result['new_price'], wac)
        return result
    
    # CASE 3: Normal SKUs - Determine price action based on status
    combined_status = row.get('combined_status', 'No Data')
    
    # Handle 'No Data' with stocks as Critical (below on track)
    if combined_status == 'No Data' and row.get('stocks', 0) > 0:
        combined_status = 'Critical'
    
    # Get price action (increase/decrease/hold)
    action, reason = get_price_action(combined_status, yesterday_status)
    
    # Apply price action - find next tier above/below current price
    new_price, price_source = apply_price_action(current_price, action, row)
    
    result['new_price'] = new_price
    result['price_source'] = price_source
    result['price_action'] = action
    result['price_reason'] = reason
    result['new_cart_rule'] = get_initial_cart_rule(row)
    result['new_margin'] = calculate_margin(new_price, wac)
    
    return result

print("Main engine function loaded.")


In [ ]:
# =============================================================================
# EXECUTE MODULE 2
# =============================================================================
print(f"Processing {len(df)} SKUs...")
print("="*60)

results = []
for idx, row in df.iterrows():
    result = generate_initial_price_push(row)
    results.append(result)
    
    if (idx + 1) % 10000 == 0:
        print(f"Processed {idx + 1}/{len(df)} SKUs...")

df_results = pd.DataFrame(results)
print(f"\n✅ Processed {len(df_results)} SKUs")


In [ ]:
# =============================================================================
# SUMMARY
# =============================================================================
print("="*60)
print("MODULE 2 SUMMARY")
print("="*60)

print(f"\nTotal SKUs processed: {len(df_results)}")
print(f"\nBy ABC Class:")
print(df_results['abc_class'].value_counts().to_string())
print(f"\nBy Price Source:")
print(df_results['price_source'].value_counts().to_string())
print(f"\nZero Demand SKUs with discount: {len(df_results[df_results['sku_discount'] > 0])}")

# Price change analysis
df_results['price_change'] = df_results['new_price'] - df_results['current_price']
df_results['price_change_pct'] = (df_results['price_change'] / df_results['current_price'] * 100).round(2)

print(f"\nPrice Change Distribution:")
print(f"  Increases: {len(df_results[df_results['price_change'] > 0])}")
print(f"  Decreases: {len(df_results[df_results['price_change'] < 0])}")
print(f"  No change: {len(df_results[df_results['price_change'] == 0])}")
print(f"\nAvg price change: {df_results['price_change_pct'].mean():.2f}%")


In [ ]:
# =============================================================================
# EXPORT RESULTS
# =============================================================================
output_cols = [
    'product_id', 'warehouse_id', 'sku', 'brand', 'cat', 'abc_class', 'sensitivity',
    'stocks', 'zero_demand', 'combined_status', 'yesterday_status',
    'current_price', 'new_price', 'price_change', 'price_change_pct',
    'wac_p', 'current_margin', 'new_margin',
    'new_cart_rule', 'sku_discount', 'price_action', 'price_source', 'price_reason'
]

# Filter to only columns that exist
output_cols = [c for c in output_cols if c in df_results.columns]

df_results[output_cols].to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Results exported to: {OUTPUT_FILE}")
print(f"Total records: {len(df_results)}")
